## 한글 문서의 분류
다음무비(http://movie.daum.net)로부터 crawl한 영화리뷰를 이용하여 분류 연습<br>
영화리뷰와 영화의 제목을 학습해서 주어진 리뷰내용으로 어떤 영화에 대한 리뷰인지를 예측하고자 함
### data file 내용
'신과함께', '코코', '라라랜드', '인피니티 워', '곤지암' 다섯개의 영화에 대해 총 1827개의 리뷰를 수집
csv 파일 안에 리뷰내용, 평점, 영화이름 의 순으로 저장되어 있음

In [2]:
import csv

text = []
y = []
with open('movie_data.csv', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        #print(row)
        if row: #그 줄에 내용이 있는 경우에만
            text.append(row[0]) #영화 리뷰를 text 리스트에 추가
            y.append(row[2]) #영화이름을 text 리스트에 추가

In [3]:
print('Num of samples: {}'.format(len(text)))
print('Movie titles of reivews: {}'.format(set(y)))

Num of samples: 1827
Movie titles of reivews: {'코코', '곤지암', '신과함께', '인피니티 워', '라라랜드'}


In [4]:
from sklearn.model_selection import train_test_split

# split data and labels into a training and a test set
X_train, X_test, y_train, y_test = train_test_split(text, y, random_state=0)
# 비율을 지정하지 않으면 75:25로 분할됨

In [5]:
len(X_train) #1827의 0.75

1370

In [6]:
from konlpy.tag import Okt #konlpy에서 Twitter 형태소 분석기를 import
#from konlpy.tag import Twitter #konlpy에서 Twitter 형태소 분석기를 import
twitter_tag = Okt()
#twitter_tag = Twitter()

ModuleNotFoundError: No module named 'konlpy'

In [8]:
print(twitter_tag.morphs(X_train[1])) #둘째 리뷰에 대해 형태소 단위로 tokenize

NameError: name 'twitter_tag' is not defined

In [7]:
twitter_tag.nouns(X_train[1]) #둘째 리뷰에서 명사만 추출

NameError: name 'twitter_tag' is not defined

In [8]:
def twit_tokenizer(text): # Twitter 형태소 분석기의 명사추출함수를 tokenizer 함수로 사용
    return twitter_tag.nouns(text)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

#tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True)
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=2) #Twitter 형태소분석기에서 명사만 추출하는 함수를 tokenizer로 이용
# twit_tokenizer 대신 twitter_tag.nouns를 직접 써도 됨
# 하나의 문서에서만 출현한 단어는 쓸모가 없으므로 제외, 즉 최소 document frequency를 2로 설정

X_train_tfidf = tfidf.fit_transform(X_train) # train data 변환 -> tfidf vector
X_test_tfidf = tfidf.transform(X_test) # test data 변환 -> tfidf vector

clf = LogisticRegression() # logistic regression 분류기 선언
clf.fit(X_train_tfidf, y_train) # 분류기 학습
print('Train score', clf.score(X_train_tfidf, y_train)) # train data 예측정확도
print('Test score', clf.score(X_test_tfidf, y_test)) # test data 예측정확도
print(X_train_tfidf.shape) # 총 1156개의 명사로 이루어짐

NameError: name 'twitter_tag' is not defined

In [10]:
X_test[:10] #test data에서 앞 10개를 출력

['졸잼 최고',
 '내용, 음악 , 연기력  무엇하나 빠지는것이 없네요 특히 음악은 계속 찾아 듣게되요^^',
 '아맥2D로 느즈막히 관람.... 히어로가 많이나오지만, 이걸 꽤나 잘 버무려놓음. 뻔한스토리의 틀을 벗어나려 노력한점은 높은점수를 줄만함.... 블럭버스터액션, 영상미는 말이필요없음...... 후속편 기대됨!',
 '후반부터 쫄렸다.',
 '진짜. 솔직히 한국 공포영화중에 이렇게 소재별로인건 정말 오랜만인듯; 지들끼리 소리지르고 정신없이 우왕자왕 심지어 무섭지도않어 효과음만크고 진짜최악임ㅉㅉ',
 '소문난 잔치에 먹을거 없음..ㅜㅜ',
 'good!',
 '아 점수를 줄 수가 없네  화면은 왜그리도 흔들어 데는지........ 재미도 없고 가볍기만하고 .... 최악의 재미없는 배멀미 영화',
 '영화 보면서 펑펑물었네요~ 부모님 사랑에 대해 다시한번 생각하게 했던 영화네요^ ^',
 '슬픈 스토리지만 삶을 돌아보게 하는 영화다. 죄를 지은자는 그 벌을 고스란히 받으리라. 사회 각종범죄자들 뉘우치길 바란다.']

In [11]:
clf.predict(X_test_tfidf[:10]) # test data의 앞 10개에 대한 예측내용

NameError: name 'clf' is not defined

In [12]:
print(y_test[:10]) # test data 앞 10개의 실제 영화제목

['인피니티 워', '라라랜드', '인피니티 워', '곤지암', '곤지암', '인피니티 워', '인피니티 워', '곤지암', '신과함께', '신과함께']


### 성능을 개선하기 위한 노력

In [13]:
# morphs()는 명사 외에도 모든 형태소를 포함
print(twitter_tag.morphs(X_train[1]))

NameError: name 'twitter_tag' is not defined

In [14]:
tfidf = TfidfVectorizer(tokenizer=twitter_tag.morphs, min_df=2) # 명사 대신 모든 형태소를 사용
#tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)
print('Train score', clf.score(X_train_tfidf, y_train))
print('Test score', clf.score(X_test_tfidf, y_test))
print(X_train_tfidf.shape)
#명사만 사용한 것에 비해 train score는 상승, test score는 하락

NameError: name 'twitter_tag' is not defined

In [15]:
print(twitter_tag.pos(X_train[1], norm=True, stem=True)) #pos()는 형태소와 품사를 함께 제공

NameError: name 'twitter_tag' is not defined

In [16]:
def twit_tokenizer2(text): #전체를 다 사용하는 대신, 명사, 동사, 형용사를 사용
    target_tags = ['Noun', 'Verb', 'Adjective']
    result = []
    for word, tag in twitter_tag.pos(text, norm=True, stem=True):
        if tag in target_tags:
            result.append(word)
#            result.append('/'.join([word, tag]))
    return result

In [17]:
print(twit_tokenizer2(X_train[1])) # 사용 예

NameError: name 'twitter_tag' is not defined

In [18]:
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer2, min_df=2) #명사, 동사, 형용사를 이용하여 tfidf 생성
#tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)
print('Train score', clf.score(X_train_tfidf, y_train))
print('Test score', clf.score(X_test_tfidf, y_test))
print(X_train_tfidf.shape)
# 현재까지 중에서 test score가 가장 뛰어남

NameError: name 'twitter_tag' is not defined

In [19]:
# 모든 형태소를 다 사용하고 품사를 알 수 있도록 하면?
def twit_tokenizer3(text):
    #target_tags = ['Noun', 'Verb', 'Adjective']
    result = []
    for word, tag in twitter_tag.pos(text, norm=True, stem=True):
        result.append('/'.join([word, tag])) #단어의 품사를 구분할 수 있도록 함
    return result

In [20]:
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer3, min_df=2)
#tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)
print('Train score', clf.score(X_train_tfidf, y_train))
print('Test score', clf.score(X_test_tfidf, y_test))
print(X_train_tfidf.shape)
#성능이 오히려 떨어지고 품사 표시 없이 전체를 다 사용한 경우에 비해 train은 떨어지고, test는 올라감

NameError: name 'twitter_tag' is not defined

In [21]:
# train score가 높으므로 ridge를 쓰면 어떨까?
from sklearn.linear_model import RidgeClassifier
ridge_clf = RidgeClassifier(alpha = 1)
ridge_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))
# train score가 올라가는 현상이 벌어짐
# test score가 올라갔으나 명사, 형용사, 동사를 쓴 것보다 떨어짐

NameError: name 'X_train_tfidf' is not defined

In [22]:
#lasso를 쓰면?
from sklearn.linear_model import LogisticRegression
import numpy as np
lasso_clf = LogisticRegression(penalty='l1', solver='liblinear')
lasso_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(lasso_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(lasso_clf.score(X_test_tfidf, y_test)))
print('Used features count: {}'.format(np.sum(lasso_clf.coef_ != 0)), 'out of', X_train_tfidf.shape[1])

NameError: name 'X_train_tfidf' is not defined

In [23]:
#lsa를 쓰면?
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=239, n_iter=7, random_state=42) #압축할 component의 수 지정
svd.fit(X_train_tfidf)  
print(svd.explained_variance_ratio_)  #계산된 각 component가 설명하는 분산의 비율
print(svd.explained_variance_ratio_.sum())  #선택된 component들이 설명하는 분산의 합 -> 선택한 component의 수에 따라 달라짐
print(svd.singular_values_) 
print(svd.components_.shape)

NameError: name 'X_train_tfidf' is not defined

In [24]:
X_train_svd = svd.transform(X_train_tfidf) #선택된 component를 이용하여 2,000개의 feature로부터 feature extract (dimension reduce)
X_test_svd = svd.transform(X_test_tfidf)

from sklearn.linear_model import LogisticRegression
SVD_clf = LogisticRegression()
SVD_clf.fit(X_train_svd, y_train)
print('Train set score: {:.3f}'.format(SVD_clf.score(X_train_svd, y_train)))
print('Test set score: {:.3f}'.format(SVD_clf.score(X_test_svd, y_test)))

NameError: name 'X_train_tfidf' is not defined

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(tokenizer=twit_tokenizer2, min_df=2).fit(X_train) #tfidf와 동일하게 max_feature를 제한하여 학습
X_train_cv = cv.transform(X_train) # train set을 변환
print('Train set dimension:', X_train_cv.shape) # 36310 대신 2000이 된 것을 확인
X_test_cv = cv.transform(X_test) # test set을 변환
print('Test set dimension:', X_test_cv.shape)

from sklearn.naive_bayes import MultinomialNB
NB_clf = MultinomialNB()
NB_clf.fit(X_train_cv, y_train)
print('Train set score: {:.3f}'.format(NB_clf.score(X_train_cv, y_train)))
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_cv, y_test)))

C:\Users\User\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


NameError: name 'twitter_tag' is not defined